In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/opt/data/in_rolls/in_rolls_state_name_age_sex.csv.gz')
df

/opt/venv3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,elector_name,age,sex,state
0,PINTU DATTA,38,Male,andaman
1,SUSHILA DEVI,39,Female,andaman
2,BALA KRISHNA,29,Male,andaman
3,CHANDRA SEKHAR,45,Male,andaman
4,SIKANDER,49,Male,andaman
5,MANIMECALAI,31,Female,andaman
6,ELMER,23,Female,andaman
7,MARIANUS TOPPO,47,Male,andaman
8,WILSON,27,Male,andaman
9,JYOTHI KIRAN,42,Female,andaman


In [3]:
# Take out null elector_name and change to upper case
df.dropna(subset=['elector_name'], inplace=True)
df.elector_name = df.elector_name.str.lower()

In [4]:
# Take out null and invalid age
df.dropna(subset=['age'], inplace=True)
df.drop(df[df.age.str.len() > 3].index, inplace=True)
df.age = df.age.astype(int)
df['birth_year'] = 2017 - df.age

In [5]:
# Take out null in sex
df.dropna(subset=['sex'], inplace=True)
# Change lower case
df.sex = df.sex.str.lower()

* when we have more than 2 words, we should use the 1st word as first name
* we can ignore rows where first_name < 2 characters
* same for where last_name < 2 characters
* and we can write than when we are describing the results
* and i suppose we need to either ignore names with just 1 word though we can assume 1 word names (not 1 character) as the first name

In [6]:
df['name_wc'] = df.elector_name.apply(lambda c: len(c.split()))

In [7]:
df['first_name'] = None
df['last_name'] = None

In [8]:
df.loc[df.name_wc == 1, 'first_name'] = df.elector_name
df

,elector_name,age,sex,state,birth_year,name_wc,first_name,last_name
0,pintu datta,38,male,andaman,1979,2,None,None
1,sushila devi,39,female,andaman,1978,2,None,None
2,bala krishna,29,male,andaman,1988,2,None,None
3,chandra sekhar,45,male,andaman,1972,2,None,None
4,sikander,49,male,andaman,1968,1,sikander,None
5,manimecalai,31,female,andaman,1986,1,manimecalai,None
6,elmer,23,female,andaman,1994,1,elmer,None
7,marianus toppo,47,male,andaman,1970,2,None,None
8,wilson,27,male,andaman,1990,1,wilson,None
9,jyothi kiran,42,female,andaman,1975,2,None,None


In [9]:
df.loc[df.name_wc >= 2, 'first_name'] = df.elector_name.apply(lambda c: c.split()[0])
df

,elector_name,age,sex,state,birth_year,name_wc,first_name,last_name
0,pintu datta,38,male,andaman,1979,2,pintu,None
1,sushila devi,39,female,andaman,1978,2,sushila,None
2,bala krishna,29,male,andaman,1988,2,bala,None
3,chandra sekhar,45,male,andaman,1972,2,chandra,None
4,sikander,49,male,andaman,1968,1,sikander,None
5,manimecalai,31,female,andaman,1986,1,manimecalai,None
6,elmer,23,female,andaman,1994,1,elmer,None
7,marianus toppo,47,male,andaman,1970,2,marianus,None
8,wilson,27,male,andaman,1990,1,wilson,None
9,jyothi kiran,42,female,andaman,1975,2,jyothi,None


In [10]:
df.loc[df.name_wc >= 2, 'last_name'] = df.elector_name.apply(lambda c: c.split()[-1])
df

,elector_name,age,sex,state,birth_year,name_wc,first_name,last_name
0,pintu datta,38,male,andaman,1979,2,pintu,datta
1,sushila devi,39,female,andaman,1978,2,sushila,devi
2,bala krishna,29,male,andaman,1988,2,bala,krishna
3,chandra sekhar,45,male,andaman,1972,2,chandra,sekhar
4,sikander,49,male,andaman,1968,1,sikander,None
5,manimecalai,31,female,andaman,1986,1,manimecalai,None
6,elmer,23,female,andaman,1994,1,elmer,None
7,marianus toppo,47,male,andaman,1970,2,marianus,toppo
8,wilson,27,male,andaman,1990,1,wilson,None
9,jyothi kiran,42,female,andaman,1975,2,jyothi,kiran


In [11]:
df.loc[df.first_name.notnull() & (df.first_name.str.len() <= 2), 'first_name'] = None
df.loc[df.last_name.notnull() & (df.last_name.str.len() <= 2), 'last_name'] = None
df

,elector_name,age,sex,state,birth_year,name_wc,first_name,last_name
0,pintu datta,38,male,andaman,1979,2,pintu,datta
1,sushila devi,39,female,andaman,1978,2,sushila,devi
2,bala krishna,29,male,andaman,1988,2,bala,krishna
3,chandra sekhar,45,male,andaman,1972,2,chandra,sekhar
4,sikander,49,male,andaman,1968,1,sikander,None
5,manimecalai,31,female,andaman,1986,1,manimecalai,None
6,elmer,23,female,andaman,1994,1,elmer,None
7,marianus toppo,47,male,andaman,1970,2,marianus,toppo
8,wilson,27,male,andaman,1990,1,wilson,None
9,jyothi kiran,42,female,andaman,1975,2,jyothi,kiran


In [12]:
df[df.first_name.isnull()]

,elector_name,age,sex,state,birth_year,name_wc,first_name,last_name
77,r elizah,28,male,andaman,1989,2,None,elizah
99,re eeza begum,24,female,andaman,1993,3,None,begum
114,b keshav rao,53,male,andaman,1964,3,None,rao
129,m pandi rajamma,35,female,andaman,1982,3,None,rajamma
166,c poongodai,35,female,andaman,1982,2,None,poongodai
203,n laxmi,34,female,andaman,1983,2,None,laxmi
281,r sulakshana,48,female,andaman,1969,2,None,sulakshana
619,y polina,80,female,andaman,1937,2,None,polina
623,a kameshwari,28,female,andaman,1989,2,None,kameshwari
677,m shiva subramaniam,40,male,andaman,1977,3,None,subramaniam


In [13]:
df[df.last_name.isnull()]

,elector_name,age,sex,state,birth_year,name_wc,first_name,last_name
4,sikander,49,male,andaman,1968,1,sikander,None
5,manimecalai,31,female,andaman,1986,1,manimecalai,None
6,elmer,23,female,andaman,1994,1,elmer,None
8,wilson,27,male,andaman,1990,1,wilson,None
14,felicitus,36,female,andaman,1981,1,felicitus,None
16,amuda,31,female,andaman,1986,1,amuda,None
19,nazarene,27,female,andaman,1990,1,nazarene,None
25,srinivas,45,male,andaman,1972,1,srinivas,None
29,recheal,45,female,andaman,1972,1,recheal,None
30,manisha,25,female,andaman,1992,1,manisha,None


In [14]:
del df['name_wc']

In [15]:
df.to_csv('/opt/data/in_rolls/in_rolls_state_name_age_sex_fn_ln.csv.gz', index=False, compression='gzip')